In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from src.supabase_client import list_users

for user in list_users():
    print(user.preferences)

race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[TheoreticalTrainingSession(day=<Day.WED: 'Wed'>, session_type=<SessionType.MODERATE: 'moderate run'>), TheoreticalTrainingSession(day=<Day.SUN: 'Sun'>, session_type=<SessionType.LONG: 'long run'>), TheoreticalTrainingSession(day=<Day.THURS: 'Thurs'>, session_type=<SessionType.MODERATE: 'moderate run'>)]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_training_week=[]
race_distance=None ideal_